In [46]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


<IPython.core.display.Javascript object>

In [47]:
# default_exp evaluation

<IPython.core.display.Javascript object>

# Evaluation

This section provides convenient evaluation schemes for both Numerai Classic and Signals.
The `Evaluator` takes a `NumerFrame` is input and return a Pandas DataFrame containing metrics for each prediction column that is given.

In [48]:
# export
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from typing import Tuple, Union

from numerai_blocks.numerframe import NumerFrame, create_numerframe
from numerai_blocks.postprocessing import FeatureNeutralizer

<IPython.core.display.Javascript object>

## 0. Base

`BaseEvaluator` implements all the evaluation logic that is common for Numerai Classic and Signals. This includes:
- Mean, Standard Deviation and Sharpe for era returns.
- Max drawdown
- Annual Percentage Yield (APY)
- Mean, Standard deviation and Sharpe for [MMC (Meta Model Contribution)](https://docs.numer.ai/tournament/metamodel-contribution) returns.
- Correlation with example predictions
- Max [feature exposure](https://forum.numer.ai/t/model-diagnostics-feature-exposure/899)
- [Feature neutral](https://docs.numer.ai/tournament/feature-neutral-correlation) Mean, Standard deviation and Sharpe
- Mean, Standard Deviation and Sharpe for TB200 (Buy top 200 stocks and sell bottom 200 stocks).
- Mean, Standard Deviation and Sharpe for TB500 (Buy top 500 stocks and sell bottom 500 stocks).


In [49]:
# export
class BaseEvaluator:
    """
    Evaluation functionality that is relevant for both
    Numerai Classic and Numerai Signals.
    :param era_col: Column name pointing to eras.
    Most commonly "era" for Numerai Classic and "friday_date" for Numerai Signals.
    :param fast_mode: Will skip compute intensive metrics if set to True,
    namely max_exposure, feature neutral mean, TB200 and TB500.
    """
    def __init__(self, era_col: str = "era", fast_mode = False):
        self.era_col = era_col
        self.fast_mode = fast_mode

    def full_evaluation(self,
                        dataf: NumerFrame,
                        example_col: str,
                        pred_cols: list = None,
                        target_col: str = "target"
                        ) -> pd.DataFrame:
        """
        Perform evaluation for each prediction column in the NumerFrame
        against give target and example prediction column.
        """
        val_stats = pd.DataFrame()
        dataf = dataf.fillna(0.5)
        pred_cols = dataf.prediction_cols if not pred_cols else pred_cols
        for col in tqdm(pred_cols, desc="Evaluation: "):
            col_stats = self.evaluation_one_col(dataf=dataf, pred_col=col,
                                                target_col=target_col,
                                                example_col=example_col)
            val_stats = pd.concat([val_stats, col_stats], axis=0)
        return val_stats

    def evaluation_one_col(self, dataf: Union[pd.DataFrame, NumerFrame], pred_col: str, target_col: str, example_col: str):
        """
        Perform evaluation for one prediction column
        against given target and example prediction column.
        """
        col_stats = pd.DataFrame()
        # Compute stats
        val_corrs = self.per_era_corrs(dataf=dataf,
                                        pred_col=pred_col,
                                        target_col=target_col
                                       )
        mean, std, sharpe = self.mean_std_sharpe(era_corrs=val_corrs)
        max_drawdown = self.max_drawdown(era_corrs=val_corrs)
        apy = self.apy(era_corrs=val_corrs)
        example_corr = self.example_correlation(dataf=dataf,
                                                pred_col=pred_col,
                                                example_col=example_col
                                                )
        mmc_mean, mmc_std, mmc_sharpe = self.mmc(dataf=dataf,
                                                 pred_col=pred_col,
                                                 target_col=target_col,
                                                 example_col=example_col
                                                 )

        col_stats.loc[pred_col, "target"] = target_col
        col_stats.loc[pred_col, "mean"] = mean
        col_stats.loc[pred_col, "std"] = std
        col_stats.loc[pred_col, "sharpe"] = sharpe
        col_stats.loc[pred_col, "max_drawdown"] = max_drawdown
        col_stats.loc[pred_col, "apy"] = apy
        col_stats.loc[pred_col, "mmc_mean"] = mmc_mean
        col_stats.loc[pred_col, "mmc_std"] = mmc_std
        col_stats.loc[pred_col, "mmc_sharpe"] = mmc_sharpe
        col_stats.loc[pred_col, "corr_with_example_preds"] = example_corr

        # Compute intensive stats
        if not self.fast_mode:
            max_feature_exposure = self.max_feature_exposure(dataf=dataf, pred_col=pred_col)
            fn_mean, fn_std, fn_sharpe = self.feature_neutral_mean_std_sharpe(dataf=dataf,
                                                                              pred_col=pred_col,
                                                                              target_col=target_col
                                                                              )
            tb200_mean, tb200_std, tb200_sharpe = self.tbx_mean_std_sharpe(dataf=dataf,
                                                                           pred_col=pred_col,
                                                                           target_col=target_col,
                                                                           tb=200
                                                                           )
            tb500_mean, tb500_std, tb500_sharpe = self.tbx_mean_std_sharpe(dataf=dataf,
                                                                           pred_col=pred_col,
                                                                           target_col=target_col,
                                                                           tb=500
                                                                           )
            col_stats.loc[pred_col, "max_feature_exposure"] = max_feature_exposure
            col_stats.loc[pred_col, "feature_neutral_mean"] = fn_mean
            col_stats.loc[pred_col, "feature_neutral_std"] = fn_std
            col_stats.loc[pred_col, "feature_neutral_sharpe"] = fn_sharpe
            col_stats.loc[pred_col, "tb200_mean"] = tb200_mean
            col_stats.loc[pred_col, "tb200_std"] = tb200_std
            col_stats.loc[pred_col, "tb200_sharpe"] = tb200_sharpe
            col_stats.loc[pred_col, "tb500_mean"] = tb500_mean
            col_stats.loc[pred_col, "tb500_std"] = tb500_std
            col_stats.loc[pred_col, "tb500_sharpe"] = tb500_sharpe
        return col_stats

    def per_era_corrs(self, dataf: pd.DataFrame, pred_col: str,
                      target_col: str) -> pd.Series:
        """ Correlation between prediction and target for each era. """
        return dataf.groupby(dataf[self.era_col])\
            .apply(lambda d: self._normalize_uniform(d[pred_col].fillna(0.5))
                   .corr(d[target_col]))

    def mean_std_sharpe(self, era_corrs: pd.Series) -> Tuple[np.float64, np.float64, np.float64]:
        """
        Average, standard deviation and Sharpe ratio for
        correlations per era.
        """
        mean = pd.Series(era_corrs.mean()).item()
        std = pd.Series(era_corrs.std(ddof=0)).item()
        sharpe = mean / std
        return mean, std, sharpe

    @staticmethod
    def max_drawdown(era_corrs: pd.Series) -> np.float64:
        """ Maximum drawdown per era. """
        # Arbitrarily large window
        rolling_max = (era_corrs + 1).cumprod().rolling(window=9000,
                                                        min_periods=1).max()
        daily_value = (era_corrs + 1).cumprod()
        max_drawdown = -((rolling_max - daily_value) / rolling_max).max()
        return max_drawdown

    @staticmethod
    def apy(era_corrs: pd.Series, stake_compounding_lag: int = 4) -> np.float64:
        """
        Annual percentage yield.
        :param era_corrs: Correlation scores by era
        :param stake_compounding_lag: Compounding lag for Numerai rounds (4 for Numerai Classic)
        """
        payout_scores = era_corrs.clip(-0.25, 0.25)
        payout_daily_value = (payout_scores + 1).cumprod()
        apy = (
                      (
                              (payout_daily_value.dropna().iloc[-1])
                              ** (1 / len(payout_scores))
                      )
                      ** (52 - stake_compounding_lag)  # 52 weeks of compounding minus n for stake compounding lag
                      - 1
              ) * 100
        return apy

    def example_correlation(self, dataf: Union[pd.DataFrame, NumerFrame],
                            pred_col: str, example_col: str):
        """ Correlations with example predictions. """
        return self.per_era_corrs(dataf=dataf,
                                  pred_col=pred_col,
                                  target_col=example_col,
                                  ).mean()

    def max_feature_exposure(self, dataf: Union[pd.DataFrame, NumerFrame], pred_col: str) -> np.float64:
        """ Maximum exposure over all features. """
        max_per_era = dataf.groupby(self.era_col).apply(
            lambda d: d[dataf.feature_cols].corrwith(d[pred_col]).abs().max())
        max_feature_exposure = max_per_era.mean(skipna=True)
        return max_feature_exposure

    def feature_neutral_mean_std_sharpe(self, dataf: Union[pd.DataFrame, NumerFrame],
                             pred_col: str, target_col: str) -> Tuple[np.float64, np.float64, np.float64]:
        """
        Feature neutralized mean performance.
        More info: https://docs.numer.ai/tournament/feature-neutral-correlation
        """
        fn = FeatureNeutralizer(pred_name=pred_col,
                                proportion=1.0)
        neutralized_dataf = fn(dataf=dataf)
        neutral_corrs = self.per_era_corrs(dataf=neutralized_dataf,
                                           pred_col=f"{pred_col}_neutralized_1.0",
                                           target_col=target_col)
        mean, std, sharpe = self.mean_std_sharpe(era_corrs=neutral_corrs)
        return mean, std, sharpe

    def tbx_mean_std_sharpe(self,
                            dataf: pd.DataFrame,
                            pred_col: str,
                            target_col: str,
                            tb: int = 200
                            ) -> Tuple[np.float64, np.float64, np.float64]:
        """
        Calculate Mean, Standard deviation and Sharpe ratio
        when we focus on the x top and x bottom predictions.
        :param tb: How many of top and bottom predictions to focus on.
        TB200 and TB500 are the most common situations.
        """
        tb_val_corrs = self._score_by_date(dataf=dataf,
                                           columns=[pred_col],
                                           target=target_col,
                                           tb=tb)
        return self.mean_std_sharpe(era_corrs=tb_val_corrs)

    def mmc(self, dataf: pd.DataFrame,
            pred_col: str,
            target_col: str,
            example_col: str
            ) -> Tuple[np.float64, np.float64, np.float64]:
        """
        MMC Mean, standard deviation and Sharpe ratio.
        More info: https://docs.numer.ai/tournament/metamodel-contribution
        """
        mmc_scores = []
        corr_scores = []
        for _, x in dataf.groupby(self.era_col):
            series = self._neutralize_series(self._normalize_uniform(x[pred_col]), (x[example_col]))
            mmc_scores.append(np.cov(series, x[target_col])[0, 1] / (0.29 ** 2))
            corr_scores.append(self._normalize_uniform(x[pred_col]).corr(x[target_col]))

        val_mmc_mean = np.mean(mmc_scores)
        val_mmc_std = np.std(mmc_scores)
        corr_plus_mmcs = [c + m for c, m in zip(corr_scores, mmc_scores)]
        corr_plus_mmc_sharpe = np.mean(corr_plus_mmcs) / np.std(corr_plus_mmcs)
        return val_mmc_mean, val_mmc_std, corr_plus_mmc_sharpe

    @staticmethod
    def _neutralize_series(series, by, proportion=1.0):
        scores = series.values.reshape(-1, 1)
        exposures = by.values.reshape(-1, 1)

        # this line makes series neutral to a constant column so that it's centered and for sure gets corr 0 with exposures
        exposures = np.hstack(
            (exposures,
             np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))

        correction = proportion * (exposures.dot(
            np.linalg.lstsq(exposures, scores, rcond=None)[0]))
        corrected_scores = scores - correction
        neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
        return neutralized

    def _score_by_date(self, dataf: pd.DataFrame, columns: list, target: str, tb: int = None):
        """
        Get era correlation based on given TB (x top and bottom predictions).
        :param tb: How many of top and bottom predictions to focus on.
        TB200 is the most common situation.
        """
        unique_eras = dataf[self.era_col].unique()
        computed = []
        for u in unique_eras:
            df_era = dataf[dataf[self.era_col] == u]
            era_pred = np.float64(df_era[columns].values.T)
            era_target = np.float64(df_era[target].values.T)

            if tb is None:
                ccs = np.corrcoef(era_target, era_pred)[0, 1:]
            else:
                tbidx = np.argsort(era_pred, axis=1)
                tbidx = np.concatenate([tbidx[:, :tb], tbidx[:, -tb:]], axis=1)
                ccs = [np.corrcoef(era_target[idx], pred[idx])[0, 1] for idx, pred in zip(tbidx, era_pred)]
                ccs = np.array(ccs)
            computed.append(ccs)
        return pd.DataFrame(np.array(computed), columns=columns, index=dataf[self.era_col].unique())

    @staticmethod
    def _normalize_uniform(df: pd.DataFrame) -> pd.Series:
        """ Normalize predictions uniformly using ranks. """
        x = (df.rank(method="first") - 0.5) / len(df)
        return pd.Series(x, index=df.index)


<IPython.core.display.Javascript object>

## 1. Numerai Classic

`NumeraiClassicEvaluator` extends the base evaluation scheme with metrics specific to Numerai Classic.

In [50]:
#export
class NumeraiClassicEvaluator(BaseEvaluator):
    """ Evaluator for all metrics that are relevant in Numerai Classic. """
    def __init__(self, era_col: str = "era", fast_mode = False):
        super().__init__(era_col=era_col, fast_mode=fast_mode)

<IPython.core.display.Javascript object>

## 2. Numerai Signals

`NumeraiSignalsEvaluator` extends the base evaluation scheme with metrics specific to Numerai Signals.


In [51]:
# export
class NumeraiSignalsEvaluator(BaseEvaluator):
    """ Evaluator for all metrics that are relevant in Numerai Signals. """
    def __init__(self, era_col: str = "friday_date", fast_mode = False):
        super().__init__(era_col=era_col, fast_mode=fast_mode)

<IPython.core.display.Javascript object>

### Example usage

#### Full evaluation

In [52]:
test_dataf = create_numerframe("test_assets/mini_numerai_version_1_data.csv")
test_dataf['era'] = [0, 0, 1, 1, 2, 2, 3, 3, 4, 4]
test_dataf.loc[:, 'target'] = np.random.uniform(size=len(test_dataf))
test_dataf.loc[:, 'prediction_random'] = np.random.uniform(size=len(test_dataf))
test_dataf.loc[:, 'prediction'] = (test_dataf['prediction_random'] + np.random.uniform(low=-0.3,
                                                                                       high=0.3,
                                                                                       size=len(test_dataf))).clip(0, 1)

evaluator = NumeraiClassicEvaluator()
val_stats = evaluator.full_evaluation(dataf=test_dataf,
                                      target_col="target",
                                      pred_cols=["prediction", "prediction_random"],
                                      example_col="prediction_random"
                                      )

Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

🤖 Neutralized 'prediction' with proportion '1.0' 🤖

New neutralized column = 'prediction_neutralized_1.0'.

✅ Finished step FeatureNeutralizer. Output shape=(10, 317). Time taken for step: 
0:00:00.052833. ✅

🤖 Neutralized 'prediction_random' with proportion '1.0' 🤖

New neutralized column = 'prediction_random_neutralized_1.0'.

✅ Finished step FeatureNeutralizer. Output shape=(10, 318). Time taken for step: 
0:00:00.062392. ✅

<IPython.core.display.Javascript object>

In [53]:
val_stats

,target,mean,std,sharpe,max_drawdown,apy,mmc_mean,mmc_std,mmc_sharpe,corr_with_example_preds,max_feature_exposure,feature_neutral_mean,feature_neutral_std,feature_neutral_sharpe,tb200_mean,tb200_std,tb200_sharpe,tb500_mean,tb500_std,tb500_sharpe
prediction,target,-0.6,0.800000,-0.750000,NaN,-99.986427,1.119251e-16,3.136522e-16,-0.750000,0.6,1.0,0.2,0.979796,0.204124,-0.6,0.800000,-0.750000,-0.6,0.800000,-0.750000
prediction_random,target,-0.2,0.979796,-0.204124,NaN,-98.170061,1.618858e-17,2.545888e-16,-0.204124,1.0,1.0,0.6,0.800000,0.750000,-0.2,0.979796,-0.204124,-0.2,0.979796,-0.204124


<IPython.core.display.Javascript object>

The `Evaluator` returns a Pandas DataFrame containing metrics for each prediction column defined.
Note that any column can be used as example prediction. For practical use cases we recommend using proper example predictions (provided by Numerai) instead of random predictions.

#### Fast evaluation

`fast_mode` skips max. feature exposure, feature neutral mean, TB200 and TB500 calculations, which can take a while to compute on full Numerai datasets.

In [54]:
evaluator = NumeraiClassicEvaluator(fast_mode=True)
val_stats_fast = evaluator.full_evaluation(dataf=test_dataf,
                                           target_col="target",
                                           pred_cols=["prediction", "prediction_random"],
                                           example_col="prediction_random"
                                           )

Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [55]:
val_stats_fast

,target,mean,std,sharpe,max_drawdown,apy,mmc_mean,mmc_std,mmc_sharpe,corr_with_example_preds
prediction,target,-0.6,0.800000,-0.750000,NaN,-99.986427,1.119251e-16,3.136522e-16,-0.750000,0.6
prediction_random,target,-0.2,0.979796,-0.204124,NaN,-98.170061,1.618858e-17,2.545888e-16,-0.204124,1.0


<IPython.core.display.Javascript object>

--------------------------------------------------

In [56]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()

Converted 01_download.ipynb.
Converted 02_numerframe.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_model.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_modelpipeline.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted 10_staking.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>